<a href="https://colab.research.google.com/github/virf96/Basico/blob/main/Sistema_de_recomendaci%C3%B3n_(KNN___NB).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Recommender system

## Preparación de ambiente

### Carga de módulos

In [1]:
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.naive_bayes import MultinomialNB, GaussianNB, BernoulliNB
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor

pd.set_option('display.float_format', lambda x: '%.2f' % x)

### Funciones relevantes

In [2]:
def get_dim(param_grid):
    return np.prod([x for x in map(len, param_grid.values())])

## Preparación de datos

### Carga de datos

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
movies = pd.read_csv("/content/drive/MyDrive/datasets/movies.csv")
links = pd.read_csv("/content/drive/MyDrive/datasets/links.csv")
ratings = pd.read_csv("/content/drive/MyDrive/datasets/ratings.csv")
tags = pd.read_csv("/content/drive/MyDrive/datasets/tags.csv")

### EDA

In [5]:
movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


In [6]:
for dataset in [movies, links, ratings, tags]:
    print(dataset.shape)
    display(dataset.sample(5))

(9742, 3)


,movieId,title,genres
4586,6812,"Rookie, The (1990)",Action|Comedy|Thriller
3339,4522,Masquerade (1988),Mystery|Romance|Thriller
5732,30707,Million Dollar Baby (2004),Drama
8891,134515,BMX Bandits (1983),Adventure|Crime|Drama
7529,84615,Cedar Rapids (2011),Comedy


(9742, 3)


,movieId,imdbId,tmdbId
7147,71468,1071804,24869.00
7160,71732,902290,21910.00
4805,7160,340855,504.00
5939,34164,361693,14922.00
2793,3735,70666,9040.00


(100836, 4)


,userId,movieId,rating,timestamp
19288,125,4643,4.00,1475407893
96446,603,1350,3.00,953925657
79668,495,54503,5.00,1458635508
1150,10,6377,3.50,1455306101
48046,312,750,5.00,1043176553


(3683, 4)


,userId,movieId,tag,timestamp
2122,474,6337,gambling,1138307244
2246,474,7086,George Bernard Shaw,1137201703
1679,474,2750,nostalgia,1137443255
2084,474,6195,books,1138498655
3514,599,296,Quotable,1498456446


In [7]:
links

,movieId,imdbId,tmdbId
0,1,114709,862.00
1,2,113497,8844.00
2,3,113228,15602.00
3,4,114885,31357.00
4,5,113041,11862.00
...,...,...,...
9737,193581,5476944,432131.00
9738,193583,5914996,445030.00
9739,193585,6397426,479308.00
9740,193587,8391976,483455.00


### Ingeniería de datos

#### Rating global

In [8]:
ratings[["userId", "rating"]].groupby("userId").agg(["min", "max", "mean"])

rating          
          min  max mean
userId                 
1        1.00 5.00 4.37
2        2.00 5.00 3.95
3        0.50 5.00 2.44
4        1.00 5.00 3.56
5        1.00 5.00 3.64
...       ...  ...  ...
606      0.50 5.00 3.66
607      1.00 5.00 3.79
608      0.50 5.00 3.13
609      3.00 4.00 3.27
610      0.50 5.00 3.69

[610 rows x 3 columns]

In [9]:
aux = ratings[["userId", "rating"]].groupby("userId").agg(["min", "max", "mean"])


In [10]:
["_".join(x) for x in aux.columns]

['rating_min', 'rating_max', 'rating_mean']

In [11]:
aux.columns = ["_".join(x) for x in aux.columns]


In [12]:
ratings = ratings.merge(aux.reset_index(), on="userId", how = "left")

In [13]:
ratings

,userId,movieId,rating,timestamp,rating_min,rating_max,rating_mean
0,1,1,4.00,964982703,1.00,5.00,4.37
1,1,3,4.00,964981247,1.00,5.00,4.37
2,1,6,4.00,964982224,1.00,5.00,4.37
3,1,47,5.00,964983815,1.00,5.00,4.37
4,1,50,5.00,964982931,1.00,5.00,4.37
...,...,...,...,...,...,...,...
100831,610,166534,4.00,1493848402,0.50,5.00,3.69
100832,610,168248,5.00,1493850091,0.50,5.00,3.69
100833,610,168250,5.00,1494273047,0.50,5.00,3.69
100834,610,168252,5.00,1493846352,0.50,5.00,3.69


#### Tags

In [14]:
tags = tags[["movieId", "tag"]].groupby("movieId").agg([lambda x:" ".join(x)])

In [15]:
tags.sample(5)

,tag
,<lambda>
movieId,
8011,politics
6667,FBI
2054,Disney
1994,ghosts
318,prison Stephen King wrongful imprisonment Morg...


In [16]:
tags.columns = ["tags"]

In [17]:
vect = CountVectorizer(min_df=1, lowercase=True, max_features=30)

In [18]:
vect.fit(tags["tags"])

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=30, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [19]:
tags = tags.drop(columns=["tags"]).reset_index().join(pd.DataFrame(data = vect.transform(tags["tags"]).todense(), columns=vect.get_feature_names()))

In [20]:
tags

,movieId,action,and,atmospheric,bad,black,comedy,crime,dark,disney,ending,fi,funny,in,movie,music,netflix,provoking,queue,quirky,religion,sci,space,superhero,surreal,suspense,thought,travel,twist,visually,war
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1567,183611,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1568,184471,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1569,187593,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1570,187595,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


#### Movies (géneros)

In [21]:
movies["ls_genres"] = movies["genres"].str.split("|")
movies["n_genres"] = movies["ls_genres"].str.len()

In [22]:
movies["n_genres"]

0       5
1       3
2       2
3       3
4       1
       ..
9737    4
9738    3
9739    1
9740    2
9741    1
Name: n_genres, Length: 9742, dtype: int64

In [23]:
for i in range(movies["n_genres"].max()):
    movies[f"genre_{i}"] = movies.apply(lambda x:x["ls_genres"][i] if x["n_genres"] >= i+1 else np.nan, axis = 1)
    aux = movies[["movieId", f"genre_{i}"]].rename(columns={f"genre_{i}": "genre"})
    if i == 0:
        movies_genre = aux.copy()
    else:
        movies_genre = movies_genre.append(aux)

In [24]:
movies_genre.sort_values(by="movieId").head(20)

,movieId,genre
0,1,Adventure
0,1,Fantasy
0,1,NaN
0,1,NaN
0,1,Comedy
0,1,NaN
0,1,NaN
0,1,NaN
0,1,Animation
0,1,Children


In [25]:
movies_genre.sort_values(by = "movieId").dropna()

,movieId,genre
0,1,Adventure
0,1,Fantasy
0,1,Comedy
0,1,Animation
0,1,Children
...,...,...
9738,193583,Fantasy
9739,193585,Drama
9740,193587,Animation
9740,193587,Action


In [26]:
mgr = movies_genre.dropna().merge(ratings, on="movieId", how="right")

In [27]:
mgr

,movieId,genre,userId,rating,timestamp,rating_min,rating_max,rating_mean
0,1,Adventure,1,4.00,964982703,1.00,5.00,4.37
1,1,Animation,1,4.00,964982703,1.00,5.00,4.37
2,1,Children,1,4.00,964982703,1.00,5.00,4.37
3,1,Comedy,1,4.00,964982703,1.00,5.00,4.37
4,1,Fantasy,1,4.00,964982703,1.00,5.00,4.37
...,...,...,...,...,...,...,...,...
274475,160836,Drama,610,3.00,1493844794,0.50,5.00,3.69
274476,160836,Thriller,610,3.00,1493844794,0.50,5.00,3.69
274477,163937,Horror,610,3.50,1493848789,0.50,5.00,3.69
274478,163937,Thriller,610,3.50,1493848789,0.50,5.00,3.69


In [28]:
mgr = mgr[["userId", "genre", "rating"]].pivot_table(index="userId", columns="genre", aggfunc=["min", "max", "mean"])

In [29]:
mgr

min                   ...     mean             
                   rating                   ...   rating             
genre  (no genres listed) Action Adventure  ... Thriller  War Western
userId                                      ...                      
1                     nan   2.00      2.00  ...     4.15 4.50    4.29
2                     nan   3.00      3.50  ...     3.70 4.50    3.50
3                     nan   0.50      0.50  ...     4.14 0.50     nan
4                     nan   1.00      1.00  ...     3.55 3.57    3.80
5                     nan   2.00      2.00  ...     3.56 3.33    3.00
...                   ...    ...       ...  ...      ...  ...     ...
606                   nan   0.50      1.50  ...     3.53 3.79    3.41
607                   nan   1.00      1.00  ...     4.11 4.17    4.00
608                   nan   0.50      0.50  ...     3.54 3.58    2.64
609                   nan   3.00      3.00  ...     3.29 3.50    4.00
610                   nan   0.50      0.50  ...     3.57 3.78    3.74

[610 rows x 60 columns]

In [30]:
mgr.columns = ["_".join(x) for x in mgr.columns]

In [31]:
mgr

,min_rating_(no genres listed),min_rating_Action,min_rating_Adventure,min_rating_Animation,min_rating_Children,min_rating_Comedy,min_rating_Crime,min_rating_Documentary,min_rating_Drama,min_rating_Fantasy,min_rating_Film-Noir,min_rating_Horror,min_rating_IMAX,min_rating_Musical,min_rating_Mystery,min_rating_Romance,min_rating_Sci-Fi,min_rating_Thriller,min_rating_War,min_rating_Western,max_rating_(no genres listed),max_rating_Action,max_rating_Adventure,max_rating_Animation,max_rating_Children,max_rating_Comedy,max_rating_Crime,max_rating_Documentary,max_rating_Drama,max_rating_Fantasy,max_rating_Film-Noir,max_rating_Horror,max_rating_IMAX,max_rating_Musical,max_rating_Mystery,max_rating_Romance,max_rating_Sci-Fi,max_rating_Thriller,max_rating_War,max_rating_Western,mean_rating_(no genres listed),mean_rating_Action,mean_rating_Adventure,mean_rating_Animation,mean_rating_Children,mean_rating_Comedy,mean_rating_Crime,mean_rating_Documentary,mean_rating_Drama,mean_rating_Fantasy,mean_rating_Film-Noir,mean_rating_Horror,mean_rating_IMAX,mean_rating_Musical,mean_rating_Mystery,mean_rating_Romance,mean_rating_Sci-Fi,mean_rating_Thriller,mean_rating_War,mean_rating_Western
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,nan,2.00,2.00,3.00,3.00,2.00,2.00,nan,1.00,2.00,5.00,2.00,nan,3.00,1.00,3.00,3.00,1.00,3.00,3.00,nan,5.00,5.00,5.00,5.00,5.00,5.00,nan,5.00,5.00,5.00,5.00,nan,5.00,5.00,5.00,5.00,5.00,5.00,5.00,nan,4.32,4.39,4.69,4.55,4.28,4.36,nan,4.53,4.30,5.00,3.47,nan,4.68,4.17,4.31,4.22,4.15,4.50,4.29
2,nan,3.00,3.50,nan,nan,3.00,2.00,3.00,2.00,nan,nan,3.00,3.00,nan,4.00,4.50,3.00,2.00,4.50,3.50,nan,5.00,5.00,nan,nan,5.00,5.00,5.00,5.00,nan,nan,3.00,4.50,nan,4.00,4.50,5.00,5.00,4.50,3.50,nan,3.95,4.17,nan,nan,4.00,3.80,4.33,3.88,nan,nan,3.00,3.75,nan,4.00,4.50,3.88,3.70,4.50,3.50
3,nan,0.50,0.50,0.50,0.50,0.50,0.50,nan,0.50,0.50,nan,4.00,nan,0.50,5.00,0.50,0.50,0.50,0.50,nan,nan,5.00,5.00,0.50,0.50,5.00,0.50,nan,4.50,5.00,nan,5.00,nan,0.50,5.00,0.50,5.00,5.00,0.50,nan,nan,3.57,2.73,0.50,0.50,1.00,0.50,nan,0.75,3.38,nan,4.69,nan,0.50,5.00,0.50,4.20,4.14,0.50,nan
4,nan,1.00,1.00,3.00,1.00,1.00,1.00,3.00,1.00,1.00,2.00,4.00,3.00,1.00,1.00,1.00,1.00,1.00,1.00,2.00,nan,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00,3.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00,nan,3.32,3.66,4.00,3.80,3.51,3.81,4.00,3.48,3.68,4.00,4.25,3.00,4.00,3.48,3.38,2.83,3.55,3.57,3.80
5,nan,2.00,2.00,3.00,3.00,2.00,3.00,nan,1.00,3.00,nan,3.00,3.00,3.00,4.00,1.00,2.00,2.00,1.00,1.00,nan,4.00,5.00,5.00,5.00,5.00,5.00,nan,5.00,5.00,nan,3.00,5.00,5.00,4.00,5.00,3.00,5.00,5.00,5.00,nan,3.11,3.25,4.33,4.11,3.47,3.83,nan,3.80,4.14,nan,3.00,3.67,4.40,4.00,3.09,2.50,3.56,3.33,3.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,nan,0.50,1.50,2.50,1.50,0.50,0.50,3.00,0.50,2.00,3.00,1.00,2.00,2.50,2.00,1.00,1.50,0.50,2.00,2.50,nan,5.00,5.00,5.00,4.50,5.00,5.00,4.00,5.00,5.00,4.50,4.50,4.00,5.00,5.00,5.00,5.00,5.00,5.00,4.50,nan,3.18,3.50,3.71,3.45,3.57,3.65,3.80,3.79,3.60,3.81,3.35,3.06,3.73,3.79,3.74,3.56,3.53,3.79,3.41
607,nan,1.00,1.00,2.00,2.00,1.00,2.00,nan,2.00,3.00,nan,2.00,5.00,2.00,4.00,2.00,1.00,1.00,2.00,4.00,nan,5.00,5.00,5.00,5.00,5.00,5.00,nan,5.00,5.00,nan,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00,4.00,nan,3.72,3.47,3.33,3.42,3.33,3.81,nan,4.01,3.57,nan,4.11,5.00,3.60,4.65,3.52,3.25,4.11,4.17,4.00
608,nan,0.50,0.50,0.50,0.50,0.50,0.50,2.00,0.50,0.50,2.00,1.00,2.00,0.50,0.50,0.50,0.50,0.50,1.50,1.00,nan,5.00,5.00,4.50,4.50,5.00,5.00,4.00,5.00,5.00,5.00,5.00,5.00,4.50,5.00,5.00,5.00,5.00,5.00,4.00,nan,3.33,3.22,3.12,2.46,2.74,3.61,3.00,3.44,3.00,3.75,3.32,4.00,2.76,3.55,2.89,3.30,3.54,3.58,2.64


In [32]:
aux = mgr.isnull().mean().sort_values().reset_index()

In [33]:
aux

,index,0
0,mean_rating_Drama,0.00
1,max_rating_Drama,0.00
2,min_rating_Drama,0.00
3,min_rating_Thriller,0.00
4,mean_rating_Thriller,0.00
5,max_rating_Thriller,0.00
6,max_rating_Comedy,0.00
7,mean_rating_Comedy,0.00
8,min_rating_Comedy,0.00
9,mean_rating_Action,0.00


In [34]:
ls_keep = aux.loc[aux[0] <= 0.35, "index"].tolist()

In [35]:
mgr = mgr[ls_keep]

In [36]:
ratings = ratings.merge(mgr.reset_index(), on="userId", how = "left")

In [37]:
ratings

,userId,movieId,rating,timestamp,rating_min,rating_max,rating_mean,mean_rating_Drama,max_rating_Drama,min_rating_Drama,min_rating_Thriller,mean_rating_Thriller,max_rating_Thriller,max_rating_Comedy,mean_rating_Comedy,min_rating_Comedy,mean_rating_Action,max_rating_Action,min_rating_Action,max_rating_Adventure,mean_rating_Romance,min_rating_Adventure,min_rating_Romance,mean_rating_Adventure,max_rating_Romance,min_rating_Sci-Fi,max_rating_Sci-Fi,mean_rating_Sci-Fi,mean_rating_Crime,max_rating_Crime,min_rating_Crime,mean_rating_Fantasy,max_rating_Fantasy,min_rating_Fantasy,mean_rating_Mystery,max_rating_Mystery,min_rating_Mystery,mean_rating_Children,min_rating_Children,max_rating_Children,min_rating_War,mean_rating_War,max_rating_War,max_rating_Horror,mean_rating_Horror,min_rating_Horror,min_rating_Animation,mean_rating_Animation,max_rating_Animation,max_rating_Musical,mean_rating_Musical,min_rating_Musical,max_rating_IMAX,mean_rating_IMAX,min_rating_IMAX,mean_rating_Western,max_rating_Western,min_rating_Western
0,1,1,4.00,964982703,1.00,5.00,4.37,4.53,5.00,1.00,1.00,4.15,5.00,5.00,4.28,2.00,4.32,5.00,2.00,5.00,4.31,2.00,3.00,4.39,5.00,3.00,5.00,4.22,4.36,5.00,2.00,4.30,5.00,2.00,4.17,5.00,1.00,4.55,3.00,5.00,3.00,4.50,5.00,5.00,3.47,2.00,3.00,4.69,5.00,5.00,4.68,3.00,nan,nan,nan,4.29,5.00,3.00
1,1,3,4.00,964981247,1.00,5.00,4.37,4.53,5.00,1.00,1.00,4.15,5.00,5.00,4.28,2.00,4.32,5.00,2.00,5.00,4.31,2.00,3.00,4.39,5.00,3.00,5.00,4.22,4.36,5.00,2.00,4.30,5.00,2.00,4.17,5.00,1.00,4.55,3.00,5.00,3.00,4.50,5.00,5.00,3.47,2.00,3.00,4.69,5.00,5.00,4.68,3.00,nan,nan,nan,4.29,5.00,3.00
2,1,6,4.00,964982224,1.00,5.00,4.37,4.53,5.00,1.00,1.00,4.15,5.00,5.00,4.28,2.00,4.32,5.00,2.00,5.00,4.31,2.00,3.00,4.39,5.00,3.00,5.00,4.22,4.36,5.00,2.00,4.30,5.00,2.00,4.17,5.00,1.00,4.55,3.00,5.00,3.00,4.50,5.00,5.00,3.47,2.00,3.00,4.69,5.00,5.00,4.68,3.00,nan,nan,nan,4.29,5.00,3.00
3,1,47,5.00,964983815,1.00,5.00,4.37,4.53,5.00,1.00,1.00,4.15,5.00,5.00,4.28,2.00,4.32,5.00,2.00,5.00,4.31,2.00,3.00,4.39,5.00,3.00,5.00,4.22,4.36,5.00,2.00,4.30,5.00,2.00,4.17,5.00,1.00,4.55,3.00,5.00,3.00,4.50,5.00,5.00,3.47,2.00,3.00,4.69,5.00,5.00,4.68,3.00,nan,nan,nan,4.29,5.00,3.00
4,1,50,5.00,964982931,1.00,5.00,4.37,4.53,5.00,1.00,1.00,4.15,5.00,5.00,4.28,2.00,4.32,5.00,2.00,5.00,4.31,2.00,3.00,4.39,5.00,3.00,5.00,4.22,4.36,5.00,2.00,4.30,5.00,2.00,4.17,5.00,1.00,4.55,3.00,5.00,3.00,4.50,5.00,5.00,3.47,2.00,3.00,4.69,5.00,5.00,4.68,3.00,nan,nan,nan,4.29,5.00,3.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100831,610,166534,4.00,1493848402,0.50,5.00,3.69,3.87,5.00,1.00,0.50,3.57,5.00,5.00,3.73,1.00,3.60,5.00,0.50,5.00,3.73,0.50,1.50,3.71,5.00,1.00,5.00,3.66,3.80,5.00,1.00,3.59,5.00,0.50,3.77,5.00,1.50,3.65,2.00,5.00,2.00,3.78,5.00,5.00,3.51,1.00,2.50,3.90,5.00,5.00,3.93,2.50,5.00,3.63,2.00,3.74,5.00,1.00
100832,610,168248,5.00,1493850091,0.50,5.00,3.69,3.87,5.00,1.00,0.50,3.57,5.00,5.00,3.73,1.00,3.60,5.00,0.50,5.00,3.73,0.50,1.50,3.71,5.00,1.00,5.00,3.66,3.80,5.00,1.00,3.59,5.00,0.50,3.77,5.00,1.50,3.65,2.00,5.00,2.00,3.78,5.00,5.00,3.51,1.00,2.50,3.90,5.00,5.00,3.93,2.50,5.00,3.63,2.00,3.74,5.00,1.00
100833,610,168250,5.00,1494273047,0.50,5.00,3.69,3.87,5.00,1.00,0.50,3.57,5.00,5.00,3.73,1.00,3.60,5.00,0.50,5.00,3.73,0.50,1.50,3.71,5.00,1.00,5.00,3.66,3.80,5.00,1.00,3.59,5.00,0.50,3.77,5.00,1.50,3.65,2.00,5.00,2.00,3.78,5.00,5.00,3.51,1.00,2.50,3.90,5.00,5.00,3.93,2.50,5.00,3.63,2.00,3.74,5.00,1.00
100834,610,168252,5.00,1493846352,0.50,5.00,3.69,3.87,5.00,1.00,0.50,3.57,5.00,5.00,3.73,1.00,3.60,5.00,0.50,5.00,3.73,0.50,1.50,3.71,5.00,1.00,5.00,3.66,3.80,5.00,1.00,3.59,5.00,0.50,3.77,5.00,1.50,3.65,2.00,5.00,2.00,3.78,5.00,5.00,3.51,1.00,2.50,3.90,5.00,5.00,3.93,2.50,5.00,3.63,2.00,3.74,5.00,1.00


#### TAD

In [38]:
df = ratings.drop(columns=["timestamp"]).merge(tags, on = "movieId", how ="left")
del movies, links, ratings, tags

In [39]:
df.head(5)

,userId,movieId,rating,rating_min,rating_max,rating_mean,mean_rating_Drama,max_rating_Drama,min_rating_Drama,min_rating_Thriller,mean_rating_Thriller,max_rating_Thriller,max_rating_Comedy,mean_rating_Comedy,min_rating_Comedy,mean_rating_Action,max_rating_Action,min_rating_Action,max_rating_Adventure,mean_rating_Romance,min_rating_Adventure,min_rating_Romance,mean_rating_Adventure,max_rating_Romance,min_rating_Sci-Fi,max_rating_Sci-Fi,mean_rating_Sci-Fi,mean_rating_Crime,max_rating_Crime,min_rating_Crime,mean_rating_Fantasy,max_rating_Fantasy,min_rating_Fantasy,mean_rating_Mystery,max_rating_Mystery,min_rating_Mystery,mean_rating_Children,min_rating_Children,max_rating_Children,min_rating_War,...,max_rating_Animation,max_rating_Musical,mean_rating_Musical,min_rating_Musical,max_rating_IMAX,mean_rating_IMAX,min_rating_IMAX,mean_rating_Western,max_rating_Western,min_rating_Western,action,and,atmospheric,bad,black,comedy,crime,dark,disney,ending,fi,funny,in,movie,music,netflix,provoking,queue,quirky,religion,sci,space,superhero,surreal,suspense,thought,travel,twist,visually,war
0,1,1,4.00,1.00,5.00,4.37,4.53,5.00,1.00,1.00,4.15,5.00,5.00,4.28,2.00,4.32,5.00,2.00,5.00,4.31,2.00,3.00,4.39,5.00,3.00,5.00,4.22,4.36,5.00,2.00,4.30,5.00,2.00,4.17,5.00,1.00,4.55,3.00,5.00,3.00,...,5.00,5.00,4.68,3.00,nan,nan,nan,4.29,5.00,3.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,1,3,4.00,1.00,5.00,4.37,4.53,5.00,1.00,1.00,4.15,5.00,5.00,4.28,2.00,4.32,5.00,2.00,5.00,4.31,2.00,3.00,4.39,5.00,3.00,5.00,4.22,4.36,5.00,2.00,4.30,5.00,2.00,4.17,5.00,1.00,4.55,3.00,5.00,3.00,...,5.00,5.00,4.68,3.00,nan,nan,nan,4.29,5.00,3.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,1,6,4.00,1.00,5.00,4.37,4.53,5.00,1.00,1.00,4.15,5.00,5.00,4.28,2.00,4.32,5.00,2.00,5.00,4.31,2.00,3.00,4.39,5.00,3.00,5.00,4.22,4.36,5.00,2.00,4.30,5.00,2.00,4.17,5.00,1.00,4.55,3.00,5.00,3.00,...,5.00,5.00,4.68,3.00,nan,nan,nan,4.29,5.00,3.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
3,1,47,5.00,1.00,5.00,4.37,4.53,5.00,1.00,1.00,4.15,5.00,5.00,4.28,2.00,4.32,5.00,2.00,5.00,4.31,2.00,3.00,4.39,5.00,3.00,5.00,4.22,4.36,5.00,2.00,4.30,5.00,2.00,4.17,5.00,1.00,4.55,3.00,5.00,3.00,...,5.00,5.00,4.68,3.00,nan,nan,nan,4.29,5.00,3.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00
4,1,50,5.00,1.00,5.00,4.37,4.53,5.00,1.00,1.00,4.15,5.00,5.00,4.28,2.00,4.32,5.00,2.00,5.00,4.31,2.00,3.00,4.39,5.00,3.00,5.00,4.22,4.36,5.00,2.00,4.30,5.00,2.00,4.17,5.00,1.00,4.55,3.00,5.00,3.00,...,5.00,5.00,4.68,3.00,nan,nan,nan,4.29,5.00,3.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,1.00,0.00,0.00


In [40]:
X = df[[x for x in df.columns if x not in ["userId", "movieId", "rating",'tags']]]
y = df[["rating"]]

In [41]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.7)

#### Escalamiento de datos

In [42]:
SimpleImputer?

In [43]:
im_x = SimpleImputer(strategy="constant", fill_value=-1)

In [44]:
sc_x = MinMaxScaler()
sc_y = MinMaxScaler()

In [45]:
X_train

,rating_min,rating_max,rating_mean,mean_rating_Drama,max_rating_Drama,min_rating_Drama,min_rating_Thriller,mean_rating_Thriller,max_rating_Thriller,max_rating_Comedy,mean_rating_Comedy,min_rating_Comedy,mean_rating_Action,max_rating_Action,min_rating_Action,max_rating_Adventure,mean_rating_Romance,min_rating_Adventure,min_rating_Romance,mean_rating_Adventure,max_rating_Romance,min_rating_Sci-Fi,max_rating_Sci-Fi,mean_rating_Sci-Fi,mean_rating_Crime,max_rating_Crime,min_rating_Crime,mean_rating_Fantasy,max_rating_Fantasy,min_rating_Fantasy,mean_rating_Mystery,max_rating_Mystery,min_rating_Mystery,mean_rating_Children,min_rating_Children,max_rating_Children,min_rating_War,mean_rating_War,max_rating_War,max_rating_Horror,...,max_rating_Animation,max_rating_Musical,mean_rating_Musical,min_rating_Musical,max_rating_IMAX,mean_rating_IMAX,min_rating_IMAX,mean_rating_Western,max_rating_Western,min_rating_Western,action,and,atmospheric,bad,black,comedy,crime,dark,disney,ending,fi,funny,in,movie,music,netflix,provoking,queue,quirky,religion,sci,space,superhero,surreal,suspense,thought,travel,twist,visually,war
88568,1.00,5.00,4.12,4.19,5.00,1.00,1.00,4.18,5.00,5.00,3.88,1.00,3.91,5.00,1.00,5.00,3.98,1.00,1.00,3.97,5.00,1.00,5.00,3.93,4.35,5.00,1.00,3.29,5.00,1.00,4.29,5.00,2.00,4.00,2.00,5.00,3.00,4.55,5.00,5.00,...,5.00,3.00,2.67,2.00,4.00,4.00,4.00,4.67,5.00,3.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
36144,1.00,5.00,4.25,4.26,5.00,1.00,1.00,4.21,5.00,5.00,4.27,3.00,4.26,5.00,2.00,5.00,4.37,2.00,2.00,4.33,5.00,2.50,5.00,4.20,4.06,5.00,3.00,4.36,5.00,2.00,4.48,5.00,2.50,4.60,3.50,5.00,3.50,4.70,5.00,5.00,...,5.00,5.00,3.80,2.00,5.00,4.29,3.00,4.50,5.00,4.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
88778,0.50,5.00,4.21,4.14,5.00,0.50,0.50,4.19,5.00,5.00,4.33,0.50,4.17,5.00,0.50,5.00,3.81,0.50,0.50,4.12,5.00,0.50,5.00,4.14,4.28,5.00,0.50,4.16,5.00,0.50,4.37,5.00,0.50,4.35,0.50,5.00,3.00,4.32,5.00,5.00,...,5.00,4.50,4.11,3.00,5.00,4.30,1.50,3.50,5.00,0.50,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
45731,1.00,5.00,3.87,3.98,5.00,1.00,1.00,3.84,5.00,5.00,3.76,1.00,3.84,5.00,1.00,5.00,3.89,1.00,2.00,3.98,5.00,2.00,5.00,4.10,3.61,5.00,1.00,3.74,5.00,1.00,3.89,5.00,2.00,3.79,1.00,5.00,3.00,4.21,5.00,5.00,...,5.00,5.00,3.64,2.00,5.00,4.00,2.00,3.57,5.00,3.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
2933,1.00,5.00,2.61,2.61,5.00,1.00,1.00,2.55,5.00,5.00,2.64,1.00,2.73,5.00,1.00,5.00,2.68,1.00,1.00,2.82,5.00,1.00,5.00,2.56,2.90,5.00,1.00,2.83,5.00,1.00,2.97,5.00,1.00,2.70,1.00,5.00,2.00,3.00,4.00,4.00,...,5.00,5.00,2.84,1.00,3.00,3.00,3.00,2.55,3.00,1.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73557,0.50,5.00,3.40,3.55,5.00,0.50,0.50,3.47,5.00,5.00,3.18,0.50,3.23,5.00,0.50,5.00,3.48,1.00,1.00,3.40,5.00,0.50,5.00,3.42,3.49,5.00,1.00,3.29,5.00,0.50,3.48,5.00,1.00,3.27,1.00,5.00,1.50,3.46,5.00,5.00,...,5.00,5.00,3.40,1.00,5.00,3.94,2.50,3.27,4.50,1.50,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
91785,1.00,5.00,3.50,3.54,5.00,1.50,1.00,3.45,5.00,5.00,3.48,1.50,3.39,5.00,1.00,5.00,3.55,1.00,1.50,3.52,5.00,1.50,5.00,3.50,3.39,5.00,1.50,3.53,5.00,1.50,3.72,5.00,2.50,3.79,2.00,5.00

In [46]:
X_train = im_x.fit_transform(X_train)

In [47]:
Xs = sc_x.fit_transform(X_train)
ys = sc_y.fit_transform(y_train)

In [48]:
Xs

array([[0.11111111, 1.        , 0.76445118, ..., 0.25      , 0.25      ,
        0.5       ],
       [0.11111111, 1.        , 0.7993157 , ..., 0.25      , 0.25      ,
        0.5       ],
       [0.        , 1.        , 0.78855693, ..., 0.25      , 0.25      ,
        0.5       ],
       ...,
       [0.        , 1.        , 0.56831061, ..., 0.        , 0.        ,
        0.        ],
       [0.11111111, 1.        , 0.56800123, ..., 0.25      , 0.25      ,
        0.5       ],
       [0.        , 1.        , 0.54404551, ..., 0.25      , 0.25      ,
        0.5       ]])

In [49]:
ys

array([[1.        ],
       [0.77777778],
       [0.66666667],
       ...,
       [0.77777778],
       [1.        ],
       [0.11111111]])

## Modelado

### K-Vecinos más cercanos

#### Cross validation

In [ ]:
knn?

In [ ]:
knn = KNeighborsRegressor(n_neighbors=100)
knn.fit(Xs, ys)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
                    metric_params=None, n_jobs=None, n_neighbors=100, p=2,
                    weights='uniform')

In [ ]:
ls_scores = cross_val_score(X=Xs, y=ys, cv=4, n_jobs=-1, estimator=knn, scoring="r2")

In [ ]:
np.mean(ls_scores), np.std(ls_scores)

(0.17598657633778197, 0.003904807011753219)

#### Hyperparametrización

In [ ]:
 range(5, 105, 5)

range(5, 105, 5)

In [ ]:
param_grid = {"n_neighbors": range(5, 50, 5),
              "algorithm": ['auto', 'ball_tree', 'kd_tree', 'brute'],
              "metric": ["euclidean", "manhattan", "chebyshev", "minkowski", "seuclidean", "mahalanobis", ]}

In [ ]:
X.shape

(100836, 54)

In [ ]:
pd.to_pickle(knn, "k_vecinos.diplo")

In [ ]:
dim = get_dim(param_grid)
dim

216

In [ ]:
rs = RandomizedSearchCV(cv=4, error_score=-1000, estimator=knn, n_jobs=-1, scoring="r2", param_distributions=param_grid, verbose=5, n_iter=dim*0.1)

In [ ]:
rs.fit(Xs, ys)

Fitting 4 folds for each of 21 candidates, totalling 84 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:691: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed: 15.9min
[Parallel(n_jobs=-1)]: Done  68 tasks      | elapsed: 200.4min
[Parallel(n_jobs=-1)]: Done  84 out of  84 | elapsed: 222.0min finished


RandomizedSearchCV(cv=4, error_score=-1000,
                   estimator=KNeighborsRegressor(algorithm='auto', leaf_size=30,
                                                 metric='minkowski',
                                                 metric_params=None,
                                                 n_jobs=None, n_neighbors=100,
                                                 p=2, weights='uniform'),
                   iid='deprecated', n_iter=21.6, n_jobs=-1,
                   param_distributions={'algorithm': ['auto', 'ball_tree',
                                                      'kd_tree', 'brute'],
                                        'metric': ['euclidean', 'manhattan',
                                                   'chebyshev', 'minkowski',
                                                   'seuclidean',
                                                   'mahalanobis'],
                                        'n_neighbors': range(5, 50, 5)},
              

In [ ]:
rs.best_score_

In [ ]:
rs.best_estimator_

### Naïve Bayes

#### Cross validation

In [51]:
y_train

,rating
88568,5.00
36144,4.00
88778,3.50
45731,4.00
2933,2.00
...,...
73557,2.00
91785,3.50
63669,4.00
84412,5.00


In [52]:
y_train.describe()

,rating
count,70585.00
mean,3.50
std,1.04
min,0.50
25%,3.00
50%,3.50
75%,4.00
max,5.00


In [53]:
yc = (y_train >= 5)*1
yc_test = (y_test >= 5)*1

In [55]:
yc

,rating
88568,1
36144,0
88778,0
45731,0
2933,0
...,...
73557,0
91785,0
63669,0
84412,1


In [ ]:
nb?

In [ ]:
nb = GaussianNB()
nb.fit(Xs, yc.values.ravel())

GaussianNB(priors=None, var_smoothing=1e-09)

In [ ]:
ls_scores = cross_val_score(X=Xs, y=yc, cv=4, n_jobs=-1, estimator=nb, scoring="roc_auc")

In [ ]:
np.mean(ls_scores), np.std(ls_scores)

(0.7301915259542806, 0.004626094998410264)

In [ ]:
df

,userId,movieId,rating,rating_min,rating_max,rating_mean,mean_rating_Drama,max_rating_Drama,min_rating_Drama,min_rating_Thriller,...,sci,space,superhero,surreal,suspense,thought,travel,twist,visually,war
0,1,1,4.00,1.00,5.00,4.37,4.53,5.00,1.00,1.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,1,3,4.00,1.00,5.00,4.37,4.53,5.00,1.00,1.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,1,6,4.00,1.00,5.00,4.37,4.53,5.00,1.00,1.00,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
3,1,47,5.00,1.00,5.00,4.37,4.53,5.00,1.00,1.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00
4,1,50,5.00,1.00,5.00,4.37,4.53,5.00,1.00,1.00,...,0.00,0.00,0.00,0.00,1.00,0.00,0.00,1.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100831,610,166534,4.00,0.50,5.00,3.69,3.87,5.00,1.00,0.50,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
100832,610,168248,5.00,0.50,5.00,3.69,3.87,5.00,1.00,0.50,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
100833,610,168250,5.00,0.50,5.00,3.69,3.87,5.00,1.00,0.50,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
100834,610,168252,5.00,0.50,5.00,3.69,3.87,5.00,1.00,0.50,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [ ]:
df[(df["movieId"] == 170875) & (df["userId"] == 610)]

,userId,movieId,rating,rating_min,rating_max,rating_mean,mean_rating_Drama,max_rating_Drama,min_rating_Drama,min_rating_Thriller,...,sci,space,superhero,surreal,suspense,thought,travel,twist,visually,war
100835,610,170875,3.00,0.50,5.00,3.69,3.87,5.00,1.00,0.50,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


In [ ]:
df